## Creiamo un metodo per leggere il file.csv


In [22]:
import os
import csv
import pprint
def read_csv(file_name):
    meanings_map = {}
    dir = os.getcwd()
    with open(dir + '/' + file_name, "r") as csv_file:
        words = csv_file.readline().replace('\n','').split(',')
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            for i in range(1, 5):
                if not meanings_map.keys().__contains__(words[i]):
                    meanings_map[words[i]] = []
                meanings_map[words[i]].append(row[i]) if len(row[i]) > 0 else None
    return meanings_map

# Esempio output
#pprint.pprint(read_csv('defs.csv'))

## Creiamo una dict con l'elenco delle parole e la loro frequenza
Dal risultato di `read_csv('defs.csv')` otteniamo una dict dove per ogni termine abbiamo una lista di definizioni. 

Il risultato finale di questo step sarà una dictionary dove per ogni parola avremo una dict termine : frequenza, questa sarà usata per la ricerca del synset su wordnet tramite il principio del Genus

Per farlo 

1. Concateniamo tutte le definizioni in una unica stringa.
2. Creiamo una lista di token dell'intera lista
3. filtriamo le parole che sono alfanumerici ( per rimuovere punteggiatura) e che non siano stop_words.

Infine memorizziamo il lemma del termine e calcoliamo la frequenza di ogni parola. 



In [123]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter

meanings_map = read_csv('defs.csv')
lm = WordNetLemmatizer()
#porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
counter_map = {}
for word in meanings_map.keys():
    definitions_join = ' '.join(meanings_map.get(word))
    word_tokens = word_tokenize(definitions_join)
    list_definitio_word = [lm.lemmatize(w.lower()) for w in word_tokens if not w.lower() in stop_words and w.lower().isalnum()]
    #list_definitio_word = [porter.stem(w.lower()) for w in word_tokens if not w.lower() in stop_words and w.lower().isalnum()]
    counter_map[word] = dict(Counter(list_definitio_word))

# Esempio
pprint.pprint(counter_map)   

{'Apprehension': {'abnormal': 1,
                  'accada': 1,
                  'act': 1,
                  'agitated': 1,
                  'agitation': 2,
                  'ansia': 1,
                  'anticipation': 1,
                  'anxiety': 10,
                  'awe': 1,
                  'bad': 4,
                  'brutto': 1,
                  'cause': 1,
                  'characterized': 1,
                  'che': 1,
                  'constant': 1,
                  'derived': 1,
                  'di': 1,
                  'different': 1,
                  'discomfort': 1,
                  'disturbance': 1,
                  'emotion': 2,
                  'emotional': 1,
                  'event': 1,
                  'expectation': 1,
                  'experience': 1,
                  'fear': 10,
                  'fearful': 1,
                  'feel': 3,
                  'feeling': 6,
                  'frightened': 1,
                  'future': 1,
     

## Creiamo un metodo per semplificare l'estrazione delle k word più frequenti data una parola
Questo ci servirà per applicare il principio del Genus

Per farlo sfruttiamo il metodo sorted dove passiamo la dictionary, specifichiamo la key tramite lambda calcolo e settiamo la modalità reverse per ottenere in ordine decrescente. 

Al temrine estriamo solo le prime K parole

In [124]:
def get_most_frequent_words(term, k= 99999):
    return [k for k, v in sorted(counter_map.get(term).items(), key=lambda item: item[1], reverse=True)][0:k]

# Esempio
print("Le prime 10 parole più frequenti per Apprehension sono:")
print(get_most_frequent_words('Apprehension', 10))

Le prime 10 parole più frequenti per Apprehension sono:
['something', 'fear', 'anxiety', 'state', 'feeling', 'happen', 'bad', 'feel', 'mind', 'mental']


# Definiamo ora una serie di metodi per calcolare l'overlap
Ci serve un sistema per valutare i nostri synset per capire quale potrebbe essere quello più corretto tra gli iponimi dei nostri
K termini estratti precedentemente.

## Metodo bag of word

Il primo metodo `bag_of_words_for_synset(wn_syn)` dato un synset estrae una bag of word con le parole contenute negli esempi 
e definizioni. Il metodo di filtraggio è lo stesso dello step precedente.

Per semplificare non son stati eseguiti conteggi sulle parole ripetute anche se avrebbe potuto portare ad una maggiore accuratezza durante la fase di overlapping.

Il risultato è un set delle parole estratte

In [125]:
from nltk.corpus import wordnet as wn

def bag_of_words_for_synset(wn_syn):
    new_list = wn_syn.examples() + [wn_syn.definition()]
    sentence_concatenated = ' '.join(new_list)
    tokens = word_tokenize(sentence_concatenated)
    bag_of_words = [lm.lemmatize(w.lower()) for w in tokens if not w.lower() in stop_words and w.lower().isalnum()]
    #bag_of_words = [porter.stem(w.lower()) for w in tokens if not w.lower() in stop_words and w.lower().isalnum()]
    return dict(Counter(bag_of_words))

# Esempio
print(bag_of_words_for_synset(wn.synset('dog.n.01')))

{'dog': 1, 'barked': 1, 'night': 1, 'member': 10, 'genus': 10, 'canis': 10, 'probably': 10, 'descended': 10, 'common': 10, 'wolf': 10, 'domesticated': 10, 'man': 10, 'since': 10, 'prehistoric': 10, 'time': 10, 'occurs': 10, 'many': 10, 'breed': 10}


## Metodo overlap

Creiamo ora un metodo per calcolare l'overlap date due dict di bag of words, rispettivamente del synset e delle definizioni di partenza

Questo metodo prese due dict restituisce il numero di intersezioni, dove un'intersezione avviene se due parole han lo stesso stemma

In [126]:
def compute_overlap(dict_a, dict_b):
    return sum(dict(Counter(dict_a) & Counter(dict_b)).values())

# Esempio
print(compute_overlap(counter_map.get('Apprehension'), bag_of_words_for_synset(wn.synset('apprehension.n.01'))))

3


## Per WSD -> Lesk
Per disambiguare sarà usato l'algoritmo lesk fornito dalle librerie nltk

In [127]:
from nltk.wsd import lesk

# lesk (lista parole della frase, parola da disambiguare)
print(lesk((get_most_frequent_words('Apprehension')), 'fear','n'))

Synset('fear.n.03')


## Cerchiamo il synset data la definizione

Per ogni termine da cercare estriamo le 10 più frequenti parole usando 
```get_most_frequent_words(word,10)```
Per ognuna di esse disambiguiamo usando lesk di nltk come da esempio precedente.
Nota: in questo caso non è passato nessun K perchè preferiamo estrarre tutte le parole delle definizioni per maggiore accuratezza

A questo punto per ogni iponimo di queste 10 parole calcoliamo l'overlap sul synset disambiguato.

Il risultato sarà salvato in una dictionary nuova

In [129]:
result_dict = {}
for word in counter_map.keys():
    #print(word)
    top_frequent_terms = get_most_frequent_words(word,4)
    inner_dict = {}
    for f_word in top_frequent_terms:
        #synset = lesk(get_most_frequent_words(word), f_word, 'n')
        for synset in wn.synsets(f_word):
            #print(synset)
            if synset is not None:
                for hyp in synset.hyponyms():
                    #print("\t", hyp)
                    if not inner_dict.__contains__(hyp):
                        bag_of_word_hyp = bag_of_words_for_synset(hyp)
                        overlap = compute_overlap(counter_map.get(word), bag_of_word_hyp)
                        inner_dict[hyp] = overlap
    #print("\n\n")
    result_dict[word] = inner_dict

for word in result_dict.keys():
    print(word)
    pprint.pprint(sorted(result_dict.get(word).items(), key=lambda item: item[1], reverse=True)[0:5])
    print("\n\n\n")
    

Courage
[(Synset('physical_ability.n.01'), 5),
 (Synset('penetration.n.04'), 3),
 (Synset('stage_fright.n.01'), 3),
 (Synset('countenance.n.03'), 3),
 (Synset('take_the_bull_by_the_horns.v.01'), 3)]




Paper
[(Synset('composite_material.n.01'), 5),
 (Synset('paper.n.01'), 5),
 (Synset('packing_material.n.01'), 4),
 (Synset('literature.n.03'), 4),
 (Synset('jot_down.v.01'), 4)]




Apprehension
[(Synset('feeling.n.01'), 5),
 (Synset('condition.n.01'), 4),
 (Synset('grace.n.01'), 4),
 (Synset('situation.n.01'), 4),
 (Synset('apprehension.n.01'), 3)]




Sharpener
[(Synset('drill.n.01'), 4),
 (Synset('acuminate.v.01'), 4),
 (Synset('jaws_of_life.n.01'), 3),
 (Synset('plow.n.01'), 3),
 (Synset('upset.n.04'), 3)]




